In [1]:
from dataset import *
from constants import *
import datetime
import tensorflow as tf
from model import *

In [2]:
train_data, validation_data, test_data = get_data_generators(DATA_DIR, BATCH_SIZE, IMAGE_SIZE)

Found 83484 images belonging to 4 classes.
Found 32 images belonging to 4 classes.
Found 968 images belonging to 4 classes.


In [3]:
model = resnet_model(input_shape=INPUT_SHAPE, num_classes = len(train_data.class_indices))

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 2048)              23564800  
_________________________________________________________________
last_dense (Dense)           (None, 4)                 8196      
_________________________________________________________________
activation (Activation)      (None, 4)                 0         
Total params: 23,572,996
Trainable params: 8,196
Non-trainable params: 23,564,800
_________________________________________________________________


In [5]:
learning_rate = 1e-3
model.compile(
  #optimizer=tf.keras.optimizers.SGD(lr=0.005, momentum=0.9), 
  optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate), 
  loss=tf.keras.losses.CategoricalCrossentropy(),
  metrics=['accuracy'])

In [6]:
log_path = LOG_DIR + 'resnet50v2_retrain_8layers_lr_1e-3' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_path, histogram_freq=1)

In [7]:
checkpoint_kwargs = dict(monitor='loss', verbose=1, save_best_only=True)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(MODEL_CHECKPOINT_DIR+'resnet50v2_retrain_8layers_lr_1e-3.hdf5', **checkpoint_kwargs)

In [8]:
steps_per_epoch = train_data.samples // train_data.batch_size
validation_steps = validation_data.samples // validation_data.batch_size

fit_kwargs = dict(steps_per_epoch=steps_per_epoch, epochs=1, validation_steps=validation_steps, 
                  validation_data=validation_data, verbose=1, callbacks = [tensorboard_callback, model_checkpoint])

In [9]:
out = model.fit(train_data, **fit_kwargs)

   1/2608 [..............................] - ETA: 1s - loss: 1.4444 - accuracy: 0.2188WARNING:tensorflow:From /home/rocky/miniconda3/envs/tf/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
 936/2608 [=========>....................] - ETA: 2:30 - loss: 0.5582 - accuracy: 0.8095

KeyboardInterrupt: 

In [ ]:
model = tf.keras.models.load_model(MODEL_CHECKPOINT_DIR+'resnet50v2_retrain_8layers_lr_1e-3.hdf5')

In [ ]:
loss, acc = model.evaluate(validation_data, verbose=2)

In [ ]:
loss, acc = model.evaluate(test_data, verbose=2)